## Task 1: Financial Data Analysis Script
This notebook performs comprehensive analysis of financial data for TSLA, BND, and SPY.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Add the parent directory to the system path 
import os
import sys
sys.path.append(os.path.abspath(os.path.join('..')))

In [ ]:
from scripts.data_analysis import *

In [ ]:
# # Set plotting style
# plt.style.use('seaborn')
# sns.set_palette("husl")

In [ ]:
# 1. Data Collection
print("\n1. Data Collection")
print("-" * 40)
    
start_date = '2015-01-01'
end_date = '2024-12-31'
tickers = ['TSLA', 'BND', 'SPY']

In [ ]:
# Fetch data for all assets
data_dict = {}
for ticker in tickers:
    data_dict[ticker] = fetch_stock_data(ticker, start_date, end_date)
    print(f"Fetched data for {ticker}")

In [ ]:
data_dict

In [ ]:
# 2. Data Cleaning and Preprocessing
print("\n2. Data Cleaning and Preprocessing")
print("-" * 40)
    
cleaned_data = {}
for ticker in tickers:
    cleaned_data[ticker], missing_values = clean_data(data_dict[ticker], ticker)
    print(f"\nProcessed data for {ticker}")
    print("Missing values:")
    print(missing_values)
    print("\nBasic statistics:")
    print(cleaned_data[ticker]['Close'].describe())

In [ ]:
# 3. Price Analysis
print("\n3. Price Analysis")
print("-" * 40)
    
for ticker in tickers:
    print(f"\nAnalyzing price trends for {ticker}")
    plot_price_analysis(cleaned_data[ticker], ticker)
    plt.close()  # Close the plot to free memory

In [ ]:
# 4. Returns Analysis
print("\n4. Returns Analysis")
print("-" * 40)
    
for ticker in tickers:
    print(f"\nAnalyzing returns distribution for {ticker}")
    plot_returns_distribution(cleaned_data[ticker], ticker)
    plt.close()

In [ ]:
# 5. Risk Analysis
print("\n5. Risk Analysis")
print("-" * 40)
    
risk_metrics = {}
for ticker in tickers:
    risk_metrics[ticker] = calculate_risk_metrics(cleaned_data[ticker])
    print(f"\nRisk Metrics for {ticker}:")
    for metric, value in risk_metrics[ticker].items():
        print(f"{metric}: {value:.4f}")
        
    print(f"\nPlotting rolling volatility for {ticker}")
    plot_rolling_volatility(cleaned_data[ticker], ticker)
    plt.close()

In [ ]:
# 6. Seasonality Analysis
print("\n6. Seasonality Analysis")
print("-" * 40)
    
print("\nPerforming seasonality analysis for Tesla")
decomposition = perform_seasonality_analysis(cleaned_data['TSLA'], 'TSLA')
plt.close()

In [ ]:
# 7. Correlation Analysis
print("\n7. Correlation Analysis")
print("-" * 40)
    
correlation_matrix = analyze_correlations(cleaned_data)
print("\nCorrelation Matrix:")
print(correlation_matrix)
plt.close()

In [ ]:
# # 8. Save Processed Data
# print("\n8. Saving Processed Data")
# print("-" * 40)
    
# for ticker, df in cleaned_data.items():
#     df.to_csv(f'{ticker}_processed.csv')
#     print(f"Saved processed data for {ticker}")


# 8. Save Processed Data
print("\n8. Saving Processed Data")
print("-" * 40)

output_folder = "../data/processed/"
for ticker, df in cleaned_data.items():
    df.to_csv(f'{output_folder}{ticker}_processed.csv', index=False)
    print(f"Saved processed data for {ticker} in {output_folder}")


In [ ]:
def generate_summary_report(cleaned_data, risk_metrics, correlation_matrix):
    """
    Generate a summary report of the analysis findings.
    """
    summary = []
    
    # Price Analysis Summary
    summary.append("Price Analysis:")
    for ticker in cleaned_data.keys():
        last_price = cleaned_data[ticker]['Close'][-1]
        price_change = (last_price / cleaned_data[ticker]['Close'][0] - 1) * 100
        summary.append(f"- {ticker}: Current Price: ${last_price:.2f}, Total Return: {price_change:.2f}%")
    
    # Risk Analysis Summary
    summary.append("\nRisk Analysis:")
    for ticker, metrics in risk_metrics.items():
        summary.append(f"- {ticker}:")
        summary.append(f"  * Annual Volatility: {metrics['Annual_Volatility']*100:.2f}%")
        summary.append(f"  * Sharpe Ratio: {metrics['Sharpe_Ratio']:.2f}")
        summary.append(f"  * Maximum Drawdown: {metrics['Max_Drawdown']*100:.2f}%")
    
    # Correlation Summary
    summary.append("\nCorrelation Analysis:")
    high_corr_pairs = []
    for i in correlation_matrix.index:
        for j in correlation_matrix.columns:
            if i < j:  # Avoid duplicate pairs
                corr = correlation_matrix.loc[i, j]
                if abs(corr) > 0.5:  # Report significant correlations
                    high_corr_pairs.append(f"- {i}-{j}: {corr:.2f}")
    
    if high_corr_pairs:
        summary.append("Significant correlations:")
        summary.extend(high_corr_pairs)
    
    # Print the summary
    print("\nAnalysis Summary:")
    print("-" * 40)
    print("\n".join(summary))

In [ ]:
# 9. Generate Summary Report
print("\n9. Summary of Findings")
print("-" * 40)
    
generate_summary_report(cleaned_data, risk_metrics, correlation_matrix)